In [80]:
import os

In [81]:
%pwd

'C:\\Users\\manud\\Machine_learning_projects\\Chest Cancer Detection using DL and MLflow'

In [65]:
os.chdir("../")

In [66]:
%pwd

'C:\\Users\\manud\\Machine_learning_projects\\Chest Cancer Detection using DL and MLflow'

In [67]:
import sys
from pathlib import Path

# add the src directory to sys.path
root = Path.cwd()
src_path = root / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

In [68]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [78]:
#from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Path('C:/Users/manud/Machine_learning_projects/Chest Cancer Detection using DL and MLflow/chest_cancer_classifier/config/config.yaml'),
        params_filepath = Path("C:/Users\manud\Machine_learning_projects\Chest Cancer Detection using DL and MLflow\chest_cancer_classifier\params.yaml")):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [70]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [71]:
#Now create data_ingestion_component

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [79]:
#now we are creating a pipeline to download and extract the zip file from drive with error handling
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-09 16:58:22,625: INFO: common: yaml file: C:\Users\manud\Machine_learning_projects\Chest Cancer Detection using DL and MLflow\chest_cancer_classifier\config\config.yaml loaded successfully]
[2025-10-09 16:58:22,625: INFO: common: yaml file: C:\Users\manud\Machine_learning_projects\Chest Cancer Detection using DL and MLflow\chest_cancer_classifier\params.yaml loaded successfully]
[2025-10-09 16:58:22,629: INFO: common: created directory at: artifacts]
[2025-10-09 16:58:22,629: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-09 16:58:22,632: INFO: 1518154814: Downloading data from https://drive.google.com/file/d/1n8q3JyWlv5uBwZMVqOIUiqe2X-pv1bMw/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1n8q3JyWlv5uBwZMVqOIUiqe2X-pv1bMw
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1n8q3JyWlv5uBwZMVqOIUiqe2X-pv1bMw&confirm=t&uuid=c41700f1-043b-45e9-93bd-ec7952011790
To: C:\Users\manud\Machine_learning_projects\Chest Cancer Detection using DL and MLflow\artifacts\data_ingestion\data.zip
100%|██████████| 208M/208M [03:22<00:00, 1.03MB/s] 

[2025-10-09 17:01:50,161: INFO: 1518154814: Downloaded data from https://drive.google.com/file/d/1n8q3JyWlv5uBwZMVqOIUiqe2X-pv1bMw/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
